# Mathematical Modeling Project 1

In [1]:
# Load necessary libraries
library(rpart);        # For decision trees
library(randomForest); # For random forests
library(glmnet);       # For logistic regression
library(caret);        # For model training and evaluation
# library(e1071);        # For SVM
library(tidyverse);    # For data manipulation

# Data is loaded into a dataframe called 'car_data'
data1 <- read.csv("C:/Users/swift/Desktop/train.csv")
data2 <- read.csv("C:/Users/swift/Desktop/test.csv")
car_data <- rbind(data1, data2)


randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.

Loading required package: Matrix

Loaded glmnet 4.1-8

Loading required package: ggplot2


Attaching package: 'ggplot2'


The following object is masked from 'package:randomForest':

    margin


Loading required package: lattice

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.4     ✔ tibble    3.2.1
✔ purrr     1.0.4     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::combine()  masks randomForest::combine()
✖ tidyr::expand()   masks Matrix::expand()
✖ dplyr::filter()   masks stats::filter()
✖ dplyr::lag()      masks stats::lag()
✖ purrr::lift()     masks caret::lift()
✖ ggplot2::margin() masks randomForest::margin()
✖ tidyr::pack()     masks Ma

In [2]:
# Defined a function to give summary information about the data for data wrangling purposes
data_information <- function(df_in){

    missing_counts <- sapply(df_in, function(x) sum(is.na(x)))
    info_df <- data.frame("num_missing" = missing_counts, stringsAsFactors = FALSE)
    
    info_df$"num_unique" <- sapply(df_in, function(x) length(unique(na.omit(x))))
    
    info_df$data_type <- sapply(df_in, class)
    
    info_df <- tibble::rownames_to_column(info_df, "variable")
                                   
    print(info_df)
    cat("\n\nThere are", nrow(df_in), "rows and", ncol(df_in), "columns of data\n")
}

# Defined functions to transform data types, remove missing data, remove outliers
# and add features to the data to help with interpreting the models. 
# We use functions here in the event that we test our model on a new data set we will be able
# to repeat the steps quickly.

treatment1 <- function(df_in){
    df_in$price <- as.numeric(df_in$price)
    df_in$age <- 2024 - df_in$year
    df_in <- na.omit(df_in)
    return(df_in)
}

treatment2 <- function(df_in){
    outlier_price <- quantile(df_in$price, 0.95)
    df_in <- df_in[df_in$price < outlier_price, ]
    return(df_in)
}

treatment3 <- function(df_in){
    upper_threshold <- quantile(df_in$engine_size, 0.99)
    df_in <- df_in[df_in$engine_size <= upper_threshold, ]
    return(df_in)
}

In [3]:
data_information(car_data)

                      variable num_missing num_unique data_type
1                        brand           0         26 character
2                        model           0       3790 character
3                         year           3         63   numeric
4                      mileage           3      21313   numeric
5                       engine           0       1290 character
6                  engine_size        1534         64   numeric
7                 transmission           0        206 character
8       automatic_transmission           3          2   numeric
9                    fuel_type           0         11 character
10                  drivetrain           0          6 character
11                     min_mpg        3752         55   numeric
12                     max_mpg        3752         58   numeric
13                     damaged         224          2   numeric
14                 first_owner         395          2   numeric
15              personal_using         2

In [4]:
car_data <- treatment1(car_data);
car_data <- treatment2(car_data);
car_data <- treatment3(car_data);

data_information(car_data)

Warning message in treatment1(car_data):
"NAs introduced by coercion"


                      variable num_missing num_unique data_type
1                        brand           0         25 character
2                        model           0       3050 character
3                         year           0         43   numeric
4                      mileage           0      16372   numeric
5                       engine           0        966 character
6                  engine_size           0         49   numeric
7                 transmission           0        156 character
8       automatic_transmission           0          2   numeric
9                    fuel_type           0          7 character
10                  drivetrain           0          4 character
11                     min_mpg           0         53   numeric
12                     max_mpg           0         55   numeric
13                     damaged           0          2   numeric
14                 first_owner           0          2   numeric
15              personal_using          

In [5]:
# Identify categorical columns
categorical_cols <- c("brand", "model", "engine", "transmission", 
                     "fuel_type", "drivetrain", "interior_color", "exterior_color")

# Convert binary variables to factors as well
binary_cols <- c("automatic_transmission", "damaged", "first_owner", "personal_using", 
                "turbo", "alloy_wheels", "adaptive_cruise_control", "navigation_system", 
                "power_liftgate", "backup_camera", "keyless_start", "remote_start", 
                "sunroof.moonroof", "automatic_emergency_braking", "stability_control", 
                "leather_seats", "memory_seat", "third_row_seating", 
                "apple_car_play.android_auto", "bluetooth", "usb_port", "heated_seats")

In [6]:
# Convert to factors
car_data[categorical_cols] <- lapply(car_data[categorical_cols], factor)
car_data[binary_cols] <- lapply(car_data[binary_cols], factor)

# Create a complete list of levels for each factor before splitting
factor_levels <- lapply(car_data[c(categorical_cols, binary_cols)], levels)

data_information(car_data)

                      variable num_missing num_unique data_type
1                        brand           0         25    factor
2                        model           0       3050    factor
3                         year           0         43   numeric
4                      mileage           0      16372   numeric
5                       engine           0        966    factor
6                  engine_size           0         49   numeric
7                 transmission           0        156    factor
8       automatic_transmission           0          2    factor
9                    fuel_type           0          7    factor
10                  drivetrain           0          4    factor
11                     min_mpg           0         53   numeric
12                     max_mpg           0         55   numeric
13                     damaged           0          2    factor
14                 first_owner           0          2    factor
15              personal_using          

In [7]:
# Split the data 
# set.seed(123) # For reproducibility
train_index <- createDataPartition(car_data$first_owner, p = 0.7, list = FALSE)
train_data <- car_data[train_index, ]
test_data <- car_data[-train_index, ]

In [8]:
# Ensure test data has the same factor levels as train data
for(col in c(categorical_cols, binary_cols)) {
  # Get all possible levels from original data
  all_levels <- factor_levels[[col]]
  
  # Ensure train data has all levels
  train_data[[col]] <- factor(train_data[[col]], levels = all_levels)
  
  # Ensure test data has all levels
  test_data[[col]] <- factor(test_data[[col]], levels = all_levels)
}

In [9]:
# Select a subset of important features to make models simpler
# Excluded 'model' and 'transmission' since it has too many levels and might cause issues
features <- c("brand", "year", "mileage", "engine_size", 
              "fuel_type", "drivetrain", "min_mpg", "damaged", "max_mpg",
              "turbo", "price", "age")

# Formula for classification (predicting first_owner)
formula <- as.formula(paste("first_owner ~", paste(features, collapse = " + ")))

In [10]:
# 1. Decision Tree
tree_model <- rpart(formula, data = train_data, method = "class")
tree_pred <- predict(tree_model, test_data, type = "class")
tree_conf <- confusionMatrix(tree_pred, test_data$first_owner)

In [11]:
# 2. Random Forest
rf_model <- randomForest(formula, data = train_data)
rf_pred <- predict(rf_model, test_data)
rf_conf <- confusionMatrix(rf_pred, test_data$first_owner)

In [12]:
# 3. Logistic Regression
# We need to create a model matrix for glmnet
x_train <- model.matrix(formula, train_data)[,-1] # Remove intercept
y_train <- train_data$first_owner
x_test <- model.matrix(formula, test_data)[,-1]
y_test <- test_data$first_owner

In [13]:
# Fit logistic regression
glm_model <- glmnet(x_train, y_train, family = "binomial", alpha = 0)
# Choose lambda using cross-validation
cv_glm <- cv.glmnet(x_train, y_train, family = "binomial", alpha = 0)
# Make predictions
glm_pred_prob <- predict(glm_model, x_test, s = cv_glm$lambda.min, type = "response")
glm_pred <- factor(ifelse(glm_pred_prob > 0.5, 1, 0), levels = levels(y_test))
glm_conf <- confusionMatrix(glm_pred, y_test)

In [14]:
# Compare model performances
print("Decision Tree Performance:")
print(tree_conf)

print("Random Forest Performance:")
print(rf_conf)

print("Logistic Regression Performance:")
print(glm_conf)

[1] "Decision Tree Performance:"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 1975  619
         1  686 2037
                                          
               Accuracy : 0.7546          
                 95% CI : (0.7428, 0.7661)
    No Information Rate : 0.5005          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.5091          
                                          
 Mcnemar's Test P-Value : 0.0677          
                                          
            Sensitivity : 0.7422          
            Specificity : 0.7669          
         Pos Pred Value : 0.7614          
         Neg Pred Value : 0.7481          
             Prevalence : 0.5005          
         Detection Rate : 0.3715          
   Detection Prevalence : 0.4879          
      Balanced Accuracy : 0.7546          
                                          
       'Positive' Class : 0       

In [15]:
# Create a comparison table
model_comparison <- data.frame(
  Model = c("Decision Tree", "Random Forest", "Logistic Regression"),
  Accuracy = c(tree_conf$overall["Accuracy"], 
               rf_conf$overall["Accuracy"], 
               glm_conf$overall["Accuracy"]),
  Sensitivity = c(tree_conf$byClass["Sensitivity"], 
                  rf_conf$byClass["Sensitivity"], 
                  glm_conf$byClass["Sensitivity"]),
  Specificity = c(tree_conf$byClass["Specificity"], 
                  rf_conf$byClass["Specificity"], 
                  glm_conf$byClass["Specificity"])
)

print(model_comparison)

                Model  Accuracy Sensitivity Specificity
1       Decision Tree 0.7545608   0.7422022   0.7669428
2       Random Forest 0.7491066   0.7497182   0.7484940
3 Logistic Regression 0.7447809   0.7068771   0.7827560
